In [ ]:
import os
import path
import re
import math
import pandas as pd
import numpy as np
import tensorflow as tf
print('tensorflow version:',tf.__version__)

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
flist_th = []
flist_tr = []
tot_rec = 0
threat_rec = 0
traffic_rec = 0
discard_rec = 0
with open("palo_alto_sample_300.txt","r") as file:
    file_th = open("threat_file.txt","a")
    file_th.truncate(0)
    file_tr = open("traffic_file.txt","a")
    file_tr.truncate(0)
    for fline in file.readlines():
        tot_rec+=1
        fline = re.sub(r'[^a-zA-Z0-9\n\/.:,_\s-]','',fline).lower()
        flist = fline.split(",")
        if (flist[3] == 'threat'):
            flist_th.append(flist)
            file_th.write(fline)
            threat_rec+=1
        elif(flist[3] == 'traffic'):
            flist_tr.append(flist)
            file_tr.write(fline)
            traffic_rec+=1
        else:
            discard_rec+=1
file_th.close()
file_tr.close()

In [ ]:
print("Total records:",tot_rec)
print("Threat records:",threat_rec)
print("Traffic records:",traffic_rec)
print("Discarded records:",discard_rec)

In [ ]:
threat_tag = ['Header','Received Time','Serial Number','Log Type','Subtype','FUTURE USE','Generated Time','Source IP',
              'Destination IP','NAT Source IP','NAT Destination IP','Rule Name','Source User','Destination User',
              'Application','Virtual system','Source zone','Destination zone','Ingress Interface','Egress Interface',
              'Log forwarding profile','FUTURE USE','Session ID','Repeat count','Source Port','Destination Port',
              'NAT Source Port','NAT Destination Port','Flags','IP Protocol','Action','Miscellaneous','Threat ID',
              'Category','Severity','Direction','Sequence Number','Action Flags','Source Location','Destination Location ',
              'FUTURE USE','Content Type','PCAP_ID','Filedigest','Cloud','URL Index','User Agent','File Type',
              'X-Forwarded-For','Referer','Sender','Subject','Recepient','Report ID','Device Group Hierarchy Level 1',
              'Device Group Hierarchy Level 2','Device Group Hierarchy Level 3','Device Group Hierarchy Level 4',
              'Virtual System Name','Device Name','FUTURE USE']

threat_tag_ = ['HDR-M HDR-D HDR-T HDR-IP HDR-M HDR-D HDR-T HDR-DN HDR-CNT','RCVD-DT RCVD-TM','SRNO','LTYP','SBTYP','FTRUSE',
              'GNRT-DT GNRT-TM','SRCIP','DSTIP','NATSIP','NATDIP','RULNM','SUSR','DUSR']

threat_df = pd.DataFrame(columns=['Sentence_Tag_Pair'])

with open("threat_file.txt","r") as file_th:
    l_ctr = 0
    for line in file_th:
        line_lst = line.split(",")
        line_list=[]
        for i in range(0,len(threat_tag_)):
            line_tup = (line_lst[i],threat_tag_[i])
            line_list.append(line_tup)
        threat_df['Sentence_Tag_Pair'] = threat_df['Sentence_Tag_Pair'].astype('object')
        threat_df.at[l_ctr,'Sentence_Tag_Pair'] = line_list
        l_ctr+= 1

In [ ]:
traffic_tag = ['Header','Received Time','Serial Number','Log Type','Subtype','FUTURE USE','Generated Time',
                'Source IP','Destination IP','NAT Source IP','NAT Destination IP','Rule Name','Source User',
                'Destination User','Application','Virtual system','Source zone','Destination zone','Ingress Interface',
                'Egress Interface','Log forwarding profile','FUTURE USE','Session ID','Repeat count','Source Port',
                'Destination Port','NAT Source Port','NAT Destination Port','Flags','IP Protocol','Action',
                'Bytes','Bytes Sent','Bytes Received','Packets','Start Time','Elapse Time','Category',
                'FUTURE USE','Sequence Number','Action Flag','Source Location','Destination Location',
                'FUTURE USE','Packets Sent','Packets Received','Session End Reason',
                'Device Group Hierarchy Level 1','Device Group Hierarchy Level 2',
                'Device Group Hierarchy Level 3','Device Group Hierarchy Level 4','Virtual Sytem name',
                'Device Name','Action Source']

traffic_tag_ = ['HDR-M HDR-D HDR-T HDR-IP HDR-M HDR-D HDR-T HDR-DN HDR-CNT','RCVD-DT RCVD-TM','SRNO','LTYP','SBTYP','FTRUSE',
              'GNRT-DT GNRT-TM','SRCIP','DSTIP','NATSIP','NATDIP','RULNM','SUSR','DUSR']

traffic_df = pd.DataFrame(columns=['Sentence_Tag_Pair'])

with open("traffic_file.txt","r") as file_tr:
    l_ctr = 0
    for line in file_tr:
        line_lst = line.split(",")
        line_list=[]
        for i in range(0,len(traffic_tag_)):
            line_tup = (line_lst[i],traffic_tag_[i])
            line_list.append(line_tup)
        traffic_df['Sentence_Tag_Pair'] = traffic_df['Sentence_Tag_Pair'].astype('object')
        traffic_df.at[l_ctr,'Sentence_Tag_Pair'] = line_list
        l_ctr+= 1

In [ ]:
df = pd.concat([threat_df,traffic_df],ignore_index=True)
df.head()

In [ ]:
df.Sentence_Tag_Pair[0]

In [ ]:
df['Sentence']=df['Sentence_Tag_Pair'].apply(lambda sentence:" ".join([s[0] for s in sentence if s[0] != '']))

df['Tag']=df['Sentence_Tag_Pair'].apply(lambda sentence:" ".join([s[1] for s in sentence if s[0] != '']))

In [ ]:
df['tokenised_sentences']=df['Sentence'].apply(lambda x:x.split())
df['tag_list']=df['Tag'].apply(lambda x:x.split())

In [ ]:
df['len_sentence']=df['tokenised_sentences'].apply(lambda x:len(x))
df['len_tag']=df['tag_list'].apply(lambda x:len(x))
df['is_equal']=df.apply(lambda row:1 if row['len_sentence']==row['len_tag'] else 0,axis=1)
df['is_equal'].value_counts()

In [ ]:
df.head()

In [ ]:
print(df.shape)
df=df[df['is_equal']!=0]
print(df.shape)

In [ ]:
sentences_list=df['Sentence'].tolist()
tags_list=df['tag_list'].tolist()

print("Number of Sentences in the Data ",len(sentences_list))
print("Are number of Sentences and Tag list equal? ",len(sentences_list)==len(tags_list))

In [ ]:
print(sentences_list)


Tokenising the sentences

We have to create a vocab of words and then each word should be assigned an unique identifier. Keras provides an Tokeniser API, which can be used for this purpose. It will both tokenise and encode sentences.

Also, since different sentences are of different length, we will have to pad the input sequence to the largest sentence length.Keras provides pad_sequences function to achieve the same

In [ ]:
tokeniser= tf.keras.preprocessing.text.Tokenizer(lower=False,filters='')
tokeniser.fit_on_texts(sentences_list)

In [ ]:
word_index = tokeniser.word_index
vocab_size = len(word_index) +1
print("Vocab size of Tokeniser ",vocab_size) ## Adding one since 0 is reserved for padding

In [ ]:
print(word_index)

In [ ]:
#Adding a space into word_index 
#this can be later used in testing, when the new word is not found in the word_index
#word_index[" "] = max(word_index.values())+1
#print(word_index)

In [ ]:
index_word={v: k for k, v in word_index.items()}
print(index_word)

glove_vectors = dict()
glove_dir = "C://Users//anarayan//Anaconda3//glove.6B"
glove_file = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding = "utf-8")
for line in glove_file:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:], dtype='float32')
    glove_vectors[word] = vectors
glove_file.close()

len(glove_vectors.keys())

glove_vectors.get('you').shape

glove_vectors.get('2020/09/15') #word vectors not present for date,time,ip addr, and other log literals

word_vector_matrix = np.zeros((vocab_size,100))

word_nt_fnd_cnt = 0
for word, index in word_index.items():
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
    else:
        word_nt_fnd_cnt +=1
print(word_nt_fnd_cnt)
#out of 86 words, 78 words are not present in GloVe

Cannot use GloVe vector as it doesnt have words defined for Machine Logs

#Extracting word and its weights from GloVe pre-trained vector 
#Cannot use GloVe vector as it doesnt have words defined for Machine Logs

embedding_index = {}
embedding_dim = 100
glove_dir = "C://Users//anarayan//Anaconda3//glove.6B"
glove_file = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding = "utf-8")
for line in glove_file:
    values = line.split()
    word = values[0]
    coeff = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coeff
glove_file.close()

#Cannot use GloVe vector as it doesnt have words defined for Machine Logs

embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
#Cannot use GolVe vector as it doesnt have words defined for Machine Logs

In [ ]:
#from gensim.models import Word2Vec
#word_vec_model = Word2Vec.load('word_vec_model.bin')
#print(word_vec_model)
#print(word_vec_model.wv.get_vector('sep'))

In [ ]:
embedding_index = {}
f = open(os.path.join('', 'palo_alto_300_100d.txt'),  encoding = "utf-8")
for line in f:
    #print('line :',line)
    values = line.split()
    #print('values :',values)
    word = values[0]
    #print('word :',word)
    coeff = np.asarray(values[1:], dtype='float32')
    #print('coeff :',coeff)
    embedding_index[word] = coeff
f.close()

In [ ]:
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    print('word :',word)
    try:
        embedding_vector = embedding_index.get(word)
        embedding_matrix[i] = embedding_vector
        print('embedding_vector :',embedding_matrix[i])
    except KeyError: 
        embedding_matrix[i] = np.zeros((1,embedding_dim))
print(len(embedding_matrix))

In [ ]:
embedding_index.get('sep')

In [ ]:
encoded_sentence=tokeniser.texts_to_sequences(sentences_list)
print("First Original Sentence ",sentences_list[0])
print("First Encoded Sentence ",encoded_sentence[0])
print("Is Length of Original Sentence Same as Encoded Sentence? ",len(sentences_list[0].split())==len(encoded_sentence[0]))
print("Length of First Sentence ",len(encoded_sentence[0]))

In [ ]:
unq_tag_list = []
for i in range(len(df)):
    for x in range(len(df['tag_list'][i])):
        unq_tag_list.append(df['tag_list'][i][x])

tags=list(set(unq_tag_list))
print(tags)
# Add Unknown Tag into the List
tags.append('UNKWN')
print(tags)

In [ ]:
num_tags=len(tags)
print("Number of Tags ",num_tags)

tags_map={tag:i for i,tag in enumerate(tags)}
print("Tags Map ",tags_map)

reverse_tags_map={v: k for k, v in tags_map.items()}
print("Reverse Tags Map ",reverse_tags_map)

In [ ]:
encoded_tags=[[tags_map[w] for w in tag] for tag in tags_list]
print("First Sentence :",sentences_list[0])
print('First Sentence Original Tags :',tags_list[0])
print("First Sentence Encoded Tags :",encoded_tags[0])
print("Is length of Original Tags and Encoded Tags same? ",len(tags_list[0])==len(encoded_tags[0]))
print("Length of Tags for First Sentence :",len(encoded_tags[0]))

In [ ]:
max_sentence_length=max([len(s.split()) for s in sentences_list])
print(max_sentence_length)

In [ ]:
#Need to pad sentence tokens with max tokens from sentence. During testing, the sentence can grow upto 2**7 = 128
#Calculate ceiling integer value of the sqrt of max_sequence_length 
n_sqrt = math.sqrt(max_sentence_length)
print(n_sqrt)
n_sqrt_ceil = math.ceil(n_sqrt)
print(n_sqrt_ceil)
inp_layer = 2**n_sqrt_ceil
print(inp_layer)
inp_layer = 128 #for testing needs, 128 is set as max length of a sentence
print(inp_layer)

In [ ]:
#Max Length is set in power of 2, so that padding can be done for empty token and further word embedding is consistent
max_len=inp_layer

padded_encoded_sentences=pad_sequences(maxlen=max_len,sequences=encoded_sentence,padding="post",value=0)
padded_encoded_tags=pad_sequences(maxlen=max_len,sequences=encoded_tags,padding="post",value=tags_map['UNKWN'])

print("Shape of Encoded Sentence ",padded_encoded_sentences.shape)
print("Shape of Encoded Labels ",padded_encoded_tags.shape)

print("First Encoded Sentence Without Padding ",encoded_sentence[0])
print("First Encoded Sentence with padding ",padded_encoded_sentences[0])
print("First Sentence Encoded Label without Padding ",encoded_tags[0])
print("First Sentence Encoded Label with Padding ",padded_encoded_tags[0])

In [ ]:
print(padded_encoded_sentences.shape)
padded_encoded_sentences

In [ ]:
print(padded_encoded_tags.shape)
padded_encoded_tags

In [ ]:
#Categorical encoding for tags present in each sentence = no. of sent X max_len tags X total no. of tags
#(since, total of 20 unique encodings present in dataset)
#target shape = (total_no_of _sentence,total_no_of_tags,no_of_unique_tags)
target= [to_categorical(i,num_classes = num_tags) for i in padded_encoded_tags]
print("Shape of Labels  after converting to Categorical for first sentence ",target[0].shape)

# Splitting the data into Train,Test and Validation Sets

In [ ]:
X_train,X_val_test,y_train,y_val_test = train_test_split(padded_encoded_sentences,target,test_size = 0.3,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_val_test,y_val_test,test_size = 0.2,random_state=42)

# X_train shape = (5,24)
# y_train shape = (5,24,20)
print("Input Train Data Shape ",X_train.shape)
print("Train Labels Length ",len(y_train))

# X_test shape = (1,24)
# y_test shape = (1,24,20)
print("Input Test Data Shape ",X_test.shape)
print("Test Labels Length ",len(y_test))

# X_val shape = (2,24)
# y_val shape = (2,24,20)
print("Input Validation Data Shape ",X_val.shape)
print("Validation Labels Length ",len(y_val))

In [ ]:
print("First Sentence in Training Data :\n",X_train[0]) #represents the sentence encoding using tokeniser (24,)
print("First sentence Label :\n",y_train[0]) #represents the tag encoding using tokeniser + encoding (24,20)
print("Shape of First Sentence -Train :",X_train[0].shape)
print("Shape of First Sentence Label  -Train :",y_train[0].shape)

# Building the Model using LSTM

In [ ]:
#conda install git

In [ ]:
#pip install git+https://www.github.com/keras-team/keras-contrib.git

In [ ]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional
from tensorflow.keras.utils import plot_model
from keras_contrib.layers import CRF

from numpy.random import seed
tf.random.set_seed(2)

In [ ]:
#embedding_dim=100                               #based on custome word2vec embedding
#vocab_size=len(tokeniser.word_index)+1          #total token in the dataset
#max_len=24                                      #max token in a sentence (23) + padding (1) 
#num_tags=len(tags)                              #number of tags in the dataset

print("Embedding Dimension :",embedding_dim)
print("Vocabulary Size :",vocab_size)
print("Maximum length of sentence :",max_len)
print("Unique Number of Tags :",num_tags)

In [ ]:
#LSTM architecture - 
#        Input: Sentence encoded in tokens: (?,24)
#        Embedding:  (?,24) --> (?,24,64)
#        BLSTM:  (?,24,64) --> (?,24,128)
#        LSTM:  (?,24,128) --> (?,24,256)

#        Output: (?,24,20) (20 no. of tags defined in this dataset)

input_word = Input(shape = (max_len,))

#using custom trained embedding 
#model = Embedding(input_dim=vocab_size+1,output_dim=embedding_dim,input_length=max_len)(input_word)
model = Embedding(input_dim=vocab_size,output_dim=embedding_dim,weights=[embedding_matrix],
                  input_length=max_len,trainable=False)(input_word)

model = Bidirectional(LSTM(units=embedding_dim,return_sequences=True,recurrent_dropout=0.2))(model)

model = LSTM(units=embedding_dim,return_sequences=True,recurrent_dropout=0.5)(model)

out = TimeDistributed(Dense(num_tags,activation = 'softmax'))(model)

#Trying to CRF layer -  but getting issues with model.fit 
#model = TimeDistributed(Dense(num_tags,activation = 'softmax'))(model)
#crf = CRF(num_tags) #Conditional Random Field
#out = crf(model)

model = Model(input_word,out)

model.summary()

In [ ]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])
#Cannot run CRF layer due to model.fit issue
#model.compile(optimizer= 'adam', loss=crf.loss_function, metrics=[crf.accuracy, 'accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

early_stopping =EarlyStopping(monitor='val_loss', patience=5)
best_model_weight = 'blstm_ner_model.h5'
model_checkpoint = ModelCheckpoint(best_model_weight, save_best_only=True, save_weights_only=True)
callback_list=[early_stopping, model_checkpoint]

In [ ]:
#history = model.fit(X_train,np.array(y_train),validation_data=(X_val,np.array(y_val)),batch_size = 4,epochs = 20, 
#                    callbacks=callbacks_list)
#history = model.fit(X_train,np.array(y_train),validation_data=(X_val,np.array(y_val)),batch_size = 4,epochs = 30)
history = model.fit(X_train,np.array(y_train),validation_data=(X_val,np.array(y_val)),batch_size=4,epochs=30,
                    shuffle=True,callbacks=callback_list)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("blstm-ner-model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("blstm-ner-model.h5")
print("Saved model to disk")

In [ ]:
# Saving the tokenizer using pickle
import pickle

# saving
with open('tokeniser.pickle', 'wb') as tokens:
    pickle.dump(tokeniser, tokens, protocol=pickle.HIGHEST_PROTOCOL)

# loading
#with open('tokeniser.pickle', 'rb') as tokens:
#    tokeniser = pickle.load(tokens)

#Saving the tags_map using pickle
with open('tags_map.pickle', 'wb') as tags:
    pickle.dump(tags_map, tags, protocol=pickle.HIGHEST_PROTOCOL)


# loading
#with open('tags_map.pickle', 'rb') as tags:
#    tags_map = pickle.load(tags)

#Saving the embedding_index as json
#with open("embedding_index.json", "w") as json:
#    pickle.dump(embedding_index, json)
# loading
#with open('tags_map.pickle', 'rb') as tags:
#    tags_map = pickle.load(tags)

# Model Evaluation

To evaluate the model, we will have to remove the padded portion and identify the accuracy. For this, for every test data, let us create a dataframe wuth the tokens and the actual and predicted value - and use it to calculate the metrics

In [ ]:
X_test.shape

In [ ]:
from tensorflow.keras.models import model_from_json

# load json and create model
json_file = open('blstm-ner-model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("blstm-ner-model.h5")
print("Loaded model from disk")

In [ ]:
preds=loaded_model.predict(X_test) ## Predict using model on Test Data
print(preds.shape)

In [ ]:
def evaluatePredictions(test_data,preds,actual):
    print("Shape of Test Data Array",test_data.shape)
    y_actual=np.argmax(np.array(actual),axis=2)
    print("y_actual :",y_actual)
    y_pred=np.argmax(preds,axis=2)
    print("y_pred :",y_pred)
    num_test_data=test_data.shape[0]
    print("Number of Test Data Points ",num_test_data)
    data=pd.DataFrame()
    df_list=[]
    for i in range(num_test_data):
        #print("Test Data :", test_data[i])
        test_str=list(test_data[i])
        #print("Test Str :", test_str)
        df=pd.DataFrame()
        df['test_tokens']=test_str
        df['tokens']=df['test_tokens'].apply(lambda x:tokeniser.index_word[x] if x!=0 else '<PAD>')
        df['actual_target_index']=list(y_actual[i])
        df['pred_target_index']=list(y_pred[i])
        df['actual_target_tag']=df['actual_target_index'].apply(lambda x:reverse_tags_map[x])
        df['pred_target_tag']=df['pred_target_index'].apply(lambda x:reverse_tags_map[x])
        df['id']=i+1
        df_list.append(df)
        #print("df list :", df_list)
    data=pd.concat(df_list)
    #print("data :", data)
    pred_data=data[data['tokens']!='<PAD>']
    #print("Pred data :", pred_data)
    accuracy=pred_data[pred_data['actual_target_tag']==pred_data['pred_target_tag']].shape[0]/pred_data.shape[0]
    print("Accuracy :", accuracy)
    
    
    return pred_data,accuracy


In [ ]:
pred_data,accuracy=evaluatePredictions(X_test,preds,y_test)

In [ ]:
y_pred=pred_data['pred_target_tag'].tolist()
y_actual=pred_data['actual_target_tag'].tolist()

In [ ]:
check_df = pred_data[['actual_target_tag','pred_target_tag']]
print(check_df)

In [ ]:
print(classification_report(y_actual,y_pred))

In [ ]:
###Testing Purpose
text1 = 'Oct 15 01:01:01 01.01.01.01 Oct 15 14:16:24 DC2-PA-3020.npi.int 1,2020/09/15 14:16:24,001801030168,SYSTEM,vpn,0,2020/09/15 14:16:24'
text2 = 'Sep 15 14:11:57 192.168.199.245 Sep 15 14:16:24 DC2-PA-3020.npi.int 1,2020/09/15 14:16:23,001801030168,TRAFFIC,end,2304,2020/09/15 14:16:23,192.168.198.239,208.67.222.222,107.6.5.20,208.67.222.222,Internet-Egnyte,npi\eu-support,,dns,vsys1,Transit_ASA,outside,ethernet1/5,ethernet1/1,Log_Forwarding,2020/09/15 14:16:23,110034,1,52911,53,34334,53,0x404019,udp,allow,370,121,249,2,2020/09/15 14:15:51,0,any,0,2978975507,0x0,192.168.0.0-192.168.255.255,United States,0,1,1,aged-out,0,0,0,0,,DC2-PA-3020,from-policy,,,0,,0,,N/A,0,0,0,0,6cfb47f4-da09-4589-b312-0dbff677b43d,0'
text_lst = text2.replace(",", " ")
text_list = text_lst.split(" ")

encoded_text_list=tokeniser.texts_to_sequences(text_list)
print(encoded_text_list)

for i in range(len(encoded_text_list)):
    if len(encoded_text_list[i]):
        encoded_text_list[i] = encoded_text_list[i][0]
    else:    
        encoded_text_list[i] = word_index.get(" ")

print(len(encoded_text_list))
print(encoded_text_list)
print(max_len)
for i in range(0,max_len):
     if (len(encoded_text_list) <= i):
        encoded_text_list.append(0)

print(len(encoded_text_list))
print(encoded_text_list)
print(text_list)

In [ ]:
pred_text_list = loaded_model.predict([encoded_text_list])

In [ ]:
pred_text_list.shape

In [ ]:
len(encoded_text_list)

In [ ]:
def generatePrediction(text_list,test_data,preds):
    y_pred=np.argmax(preds,axis=2)
    print("y_pred :",y_pred)
    print(text_list)
    data=pd.DataFrame()
    df_list=[]
    print(len(test_data))
    for i in range(len(test_data)):
        print("Test Data :", test_data[i])
        test_str=list(test_data[i])
        print("Test Str :", test_str)
        df=pd.DataFrame()
        text_df=pd.DataFrame()
        text_df['text_list']=text_list
        df['test_tokens']=test_str
        df['tokens_from_word_index']=df['test_tokens'].apply(lambda x:index_word[x] if x!=0 else '<PAD>')
        df['pred_target_index']=list(y_pred[i])
        df['pred_target_tag']=df['pred_target_index'].apply(lambda x:reverse_tags_map[x])
        df['id']=i+1
        df = pd.concat([text_df,df],axis=1)
        df_list.append(df)
    data=pd.concat(df_list)
    pred_data=data[data['tokens_from_word_index']!='<PAD>']
    
    
    return pred_data


In [ ]:
pred_data_text_list = generatePrediction(text_list,[encoded_text_list],pred_text_list)

In [ ]:
pred_data_text_list[0:20]